In [64]:
# ML2 Final Project
# DS4420
# Christian Peterson, Shreesh Dassarkar, Jack Roberge
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [110]:
nfl_data = pd.read_csv("/Users/Peterson/Downloads/nfl_2019.csv")
print(len(nfl_data))

C:\Users\Peterson\AppData\Local\Temp\ipykernel_5820\940112283.py:1: DtypeWarning: Columns (42,167,168,169,170,175,176,179,180,183,184,189,190,191,192,195,196,199,200,204,205,206,219,220,221,222,223,224,227,228,232,233,234,239,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl_data = pd.read_csv("/Users/Peterson/Downloads/nfl_2019.csv")


47548


In [111]:
nfl_data.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,receiver,pass,rush,name,first_down,play,passer_id,rusher_id,receiver_id,qb_epa
0,35,2019090500,CHI,GB,GB,away,CHI,CHI,35.0,2019-09-05,...,NaN,0,0,NaN,0.0,0,NaN,NaN,NaN,0.000000
1,35,2019091600,NYJ,CLE,CLE,away,NYJ,NYJ,35.0,2019-09-16,...,NaN,0,0,NaN,0.0,0,NaN,NaN,NaN,-0.589570
2,35,2019102702,CHI,LAC,LAC,away,CHI,CHI,35.0,2019-10-27,...,NaN,0,0,NaN,0.0,0,NaN,NaN,NaN,0.098560
3,35,2019111000,CHI,DET,DET,away,CHI,CHI,35.0,2019-11-10,...,NaN,0,0,NaN,0.0,0,NaN,NaN,NaN,-0.095590
4,35,2019111400,CLE,PIT,PIT,away,CLE,CLE,35.0,2019-11-14,...,NaN,0,0,NaN,0.0,0,NaN,NaN,NaN,-0.292138


In [112]:
dtypes = [nfl_data[series].dtype for series in nfl_data.columns]
for i,dtype in enumerate(dtypes):
    print(f"{nfl_data.columns[i]}={dtype}\n")

play_id=int64

game_id=int64

home_team=object

away_team=object

posteam=object

posteam_type=object

defteam=object

side_of_field=object

yardline_100=float64

game_date=object

quarter_seconds_remaining=float64

half_seconds_remaining=float64

game_seconds_remaining=float64

game_half=object

quarter_end=int64

drive=int64

sp=int64

qtr=int64

down=float64

goal_to_go=int64

time=object

yrdln=object

ydstogo=int64

ydsnet=int64

desc=object

play_type=object

yards_gained=float64

shotgun=int64

no_huddle=int64

qb_dropback=float64

qb_kneel=int64

qb_spike=int64

qb_scramble=int64

pass_length=object

pass_location=object

air_yards=float64

yards_after_catch=float64

run_location=object

run_gap=object

field_goal_result=object

kick_distance=float64

extra_point_result=object

two_point_conv_result=object

home_timeouts_remaining=int64

away_timeouts_remaining=int64

timeout=float64

timeout_team=object

td_team=object

posteam_timeouts_remaining=float64

defteam_timeouts_rema

In [113]:
bool_df = nfl_data[["pass_length", "pass_location", "run_location"]]
#~bool_df.loc[bool_df["pass_length"] == np.nan]
#print(set(bool_df["pass_length"]))
#print(set(bool_df["pass_location"]))
#print(set(bool_df["run_location"]))

# Find where data is missing in all of these
# We won't be able to classify these
missing = bool_df[bool_df["pass_length"].isna() & bool_df["run_location"].isna()]
missing

,pass_length,pass_location,run_location
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
47541,NaN,NaN,NaN
47542,NaN,NaN,NaN
47543,NaN,NaN,NaN
47545,NaN,NaN,NaN


In [114]:
# Drop that data from the dataset
check = int(len(nfl_data)) - int(len(missing))
print(check)

nfl_data = nfl_data.drop(index=missing.index)
print(len(nfl_data))

32026
32026


In [115]:
# Data cleaning / manipulation functions

def encode_x(x_df, categ_ttls):
    """
    Encode all the categorical features
    using the categorical column names by
    updating the DataFrame
    """
    for col in categ_ttls:
        categ_data = x_df[col].values.reshape(-1, 1)

        # Encode
        encoder = OneHotEncoder()
        categ_encoded = encoder.fit_transform(categ_data)

        # Handle multidimensional bools
        encoded_full = pd.DataFrame(
            categ_encoded.toarray(),
            columns=encoder.get_feature_names_out([col]),
            #index=x_df[col].index
            index=x_df.index
        )

        # Add to the df
        x_df = x_df.drop(columns=[col])
        x_df = pd.concat([x_df, encoded_full], axis=1)

    return x_df

def conditional_bool(p_dist, p_direc, r_direc):
    """
    Creates a boolean variable based on
    the pass or run distance & direction
    """
    # First determine if the entry was a run or pass
    if pd.notna(r_direc):
        return f"{r_direc.strip()} run"
        
    # We removed all unnecessary data, so if it isn't
    # a run, it should be a pass
    else:
        return f"{p_dist.strip()} {p_direc.strip()} pass"

In [116]:
# Isolate the x_features & create an x_data array
x_features = ["posteam", "side_of_field", "shotgun", "no_huddle", 
              "yrdln", "down", "goal_to_go",
              "qtr", "game_seconds_remaining", "posteam_score"]  # Note this order will be changed
x_data = nfl_data[x_features].copy()  #.values

# Modify `side_of_field` column to reflect proximity to end zone
# If we need to interpret results later on
field_sides = x_data.apply(lambda row: "Theirs" 
                                         if row["posteam"] == row["side_of_field"] 
                                       else "Opposing", axis=1)
x_data["side_of_field"] = field_sides

# Encode the categorical data   
categ_cols = ["posteam", "side_of_field", "yrdln", "down"]  # Some of the others are already encoded
x_data = encode_x(x_data, categ_cols)  # Note - still a DF

# Create the boolean target
nfl_data["pass_outcome"] = nfl_data.apply(lambda x: conditional_bool(p_dist=x["pass_length"],
                                                                     p_direc=x["pass_location"],
                                                                     r_direc=x["run_location"]), axis=1)
# x features = x_data
# y targ = "pass outcome"


In [117]:
print(x_data.iloc[0])  # THIS WILL TAKE A LONG TIME TO RUN B/C OF ENCODINGS

shotgun                      1.0
no_huddle                    0.0
goal_to_go                   0.0
qtr                          1.0
game_seconds_remaining    3600.0
                           ...  
yrdln_WAS 9                  0.0
down_1.0                     1.0
down_2.0                     0.0
down_3.0                     0.0
down_4.0                     0.0
Name: 198, Length: 1627, dtype: float64
